In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken
from matplotlib import pyplot as plt


from homework2.TransformerBlock.task import Block
from homework2.TokenizationTiktoken.task import tokenization

In [35]:
# hyper-parameters
torch.manual_seed(36)
batch_size = 16 
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda'
eval_iters = 100
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

Let's start by opening the text that our model will learn from and translating it into a set of tokens

Dividing the data

In [3]:
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
encoded_text, stoi, itos = tokenization(text)
vocab_size = len(stoi)
data = torch.tensor(encoded_text, dtype=torch.long).to(device)

In [4]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

A function that returns a batch of data

In [5]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


The most important part is actually our model, which contains everything that you have written before

In [7]:
class GPT2(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) 
        self.lm_head = nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) 
            idx_next = torch.multinomial(probs, num_samples=1) 
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx

In [8]:
model = GPT2()
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [9]:
training_loss = []
validation_loss = []
for iter in range(max_iters):
    
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        training_loss.append(losses['train'])
        validation_loss.append(losses['val'])
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
plt.plot(training_loss, label='train_loss')
plt.plot(validation_loss,label='val_loss')
plt.legend()
plt.show()

step 0: train loss 9.5655, val loss 9.5671
step 100: train loss 6.9104, val loss 7.0707
step 200: train loss 6.7317, val loss 6.9266
step 300: train loss 6.2817, val loss 6.5726
step 400: train loss 5.9259, val loss 6.3373
step 500: train loss 5.7210, val loss 6.2416
step 600: train loss 5.5542, val loss 6.0941
step 700: train loss 5.4411, val loss 6.0119
step 800: train loss 5.3482, val loss 5.9118
step 900: train loss 5.2552, val loss 5.8858
step 1000: train loss 5.1401, val loss 5.8144
step 1100: train loss 5.0792, val loss 5.7804
step 1200: train loss 5.0443, val loss 5.7302
step 1300: train loss 4.9461, val loss 5.7350
step 1400: train loss 4.9091, val loss 5.7674
step 1500: train loss 4.8559, val loss 5.6946
step 1600: train loss 4.8202, val loss 5.6715
step 1700: train loss 4.8217, val loss 5.6049
step 1800: train loss 4.7652, val loss 5.6031
step 1900: train loss 4.7083, val loss 5.6502
step 2000: train loss 4.7048, val loss 5.6642
step 2100: train loss 4.6356, val loss 5.5635


KeyboardInterrupt: 

In [43]:
encoding = tiktoken.get_encoding("cl100k_base")
s = "i remember a wonderful moment"
ids = encoding.encode(s)
context = torch.tensor([stoi[x] for x in ids], dtype=torch.long, device=device).view(1, len(ids))
predict = m.generate(context, max_new_tokens=100)[0].tolist()
print(encoding.decode([itos[i] for i in predict]))

i remember a wonderful moment,
And a world to hisuser down.

RICHARD:
With Rivers-cloth tears in the leash,
On pain of false experiencequaintges men.

GLOUCESTER:
My lord, I am a pleasant gentleman.
I mean of something king,
And do not draw her with Angelo,
The County Paris, thou hear know along!
If thy words destroy'd love a word buried thing:
Who boots notb London spoke he hate.

RICHARD:
For these brings not o'clock:


Don't be afraid of what your model gives out:)
In reality, such models are trained on a huge array of data and have an incredible number of parameters, but even our small example shows how the model captures the connection between some words